<a href="https://colab.research.google.com/github/quangnguyen-james/Datamining-Technical/blob/main/bread_basket_apriori_and_fbgrowth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import random

In [2]:
process_data = pd.read_csv('/content/bread basket.csv')
process_data.info()
#process_data.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Transaction      20507 non-null  int64 
 1   Item             20507 non-null  object
 2   date_time        20507 non-null  object
 3   period_day       20507 non-null  object
 4   weekday_weekend  20507 non-null  object
dtypes: int64(1), object(4)
memory usage: 801.2+ KB


In [3]:
#Kiểm tra duplicate
print(process_data.duplicated().sum())
# Lấy ra 10 bản ghi bị duplicate
duplicates = process_data[process_data.duplicated()].head(10)
print(duplicates)

1620
     Transaction               Item         date_time period_day  \
2              2       Scandinavian  30-10-2016 10:05    morning   
25            11              Bread  30-10-2016 10:27    morning   
49            21             Coffee  30-10-2016 10:49    morning   
101           49             Coffee  30-10-2016 12:23  afternoon   
123           60             Coffee  30-10-2016 13:14  afternoon   
127           62  Hearty & Seasonal  30-10-2016 13:24  afternoon   
130           62             Coffee  30-10-2016 13:24  afternoon   
178           85             Coffee  31-10-2016 09:10    morning   
184           88             Coffee  31-10-2016 09:16    morning   
194           94             Coffee  31-10-2016 09:39    morning   

    weekday_weekend  
2           weekend  
25          weekend  
49          weekend  
101         weekend  
123         weekend  
127         weekend  
130         weekend  
178         weekday  
184         weekday  
194         weekday  


In [4]:
# Xóa dữ liệu trùng
process_data = process_data.drop_duplicates()
#Kiểm tra trùng
print(process_data.duplicated().sum())


0


In [5]:
# Số sample còn lại
print(process_data.shape)
process_data.head(50)

(18887, 5)


,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend
5,3,Cookies,30-10-2016 10:07,morning,weekend
6,4,Muffin,30-10-2016 10:08,morning,weekend
7,5,Coffee,30-10-2016 10:13,morning,weekend
8,5,Pastry,30-10-2016 10:13,morning,weekend
9,5,Bread,30-10-2016 10:13,morning,weekend
10,6,Medialuna,30-10-2016 10:16,morning,weekend


In [6]:
#Kiểm tra transaction is null
print(process_data['Transaction'].isnull().sum())

0


In [7]:
#Đếm số loại sản phẩm (Item)
print(process_data['Item'].value_counts())
#Kiểm tra có bao nhiêu sản phẩm (Item)
print(process_data['Item'].unique())

Item
Coffee            4528
Bread             3097
Tea               1350
Cake               983
Pastry             815
                  ... 
Adjustment           1
The BART             1
Olum & polenta       1
Gift voucher         1
Raw bars             1
Name: count, Length: 94, dtype: int64
['Bread' 'Scandinavian' 'Hot chocolate' 'Jam' 'Cookies' 'Muffin' 'Coffee'
 'Pastry' 'Medialuna' 'Tea' 'Tartine' 'Basket' 'Mineral water'
 'Farm House' 'Fudge' 'Juice' "Ella's Kitchen Pouches" 'Victorian Sponge'
 'Frittata' 'Hearty & Seasonal' 'Soup' 'Pick and Mix Bowls' 'Smoothies'
 'Cake' 'Mighty Protein' 'Chicken sand' 'Coke' 'My-5 Fruit Shoot'
 'Focaccia' 'Sandwich' 'Alfajores' 'Eggs' 'Brownie' 'Dulce de Leche'
 'Honey' 'The BART' 'Granola' 'Fairy Doors' 'Empanadas' 'Keeping It Local'
 'Art Tray' 'Bowl Nic Pitt' 'Bread Pudding' 'Adjustment' 'Truffles'
 'Chimichurri Oil' 'Bacon' 'Spread' 'Kids biscuit' 'Siblings'
 'Caramel bites' 'Jammie Dodgers' 'Tiffin' 'Olum & polenta' 'Polenta'
 'The Nomad

In [8]:
#Gộp Items trong cùng 1 giao dịch
transactions = process_data.groupby('Transaction')['Item'].apply(list)
print(transactions)


Transaction
1                                                 [Bread]
2                                          [Scandinavian]
3                           [Hot chocolate, Jam, Cookies]
4                                                [Muffin]
5                                 [Coffee, Pastry, Bread]
                              ...                        
9680                                              [Bread]
9681    [Truffles, Tea, Spanish Brunch, Christmas common]
9682                  [Muffin, Tacos/Fajita, Coffee, Tea]
9683                                     [Coffee, Pastry]
9684                                          [Smoothies]
Name: Item, Length: 9465, dtype: object


In [9]:
# 2. Chuyển dữ liệu sang định dạng one-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
print("\nDataFrame one-hot encoding:")
print(df)


DataFrame one-hot encoding:
      Adjustment  Afternoon with the baker  Alfajores  Argentina Night  \
0          False                     False      False            False   
1          False                     False      False            False   
2          False                     False      False            False   
3          False                     False      False            False   
4          False                     False      False            False   
...          ...                       ...        ...              ...   
9460       False                     False      False            False   
9461       False                     False      False            False   
9462       False                     False      False            False   
9463       False                     False      False            False   
9464       False                     False      False            False   

      Art Tray  Bacon  Baguette  Bakewell  Bare Popcorn  Basket  ...  \
0        F

## Thuật toán Apriori

In [14]:
minSupport = 0.005
minConfidence = 0.1

In [16]:
# 3. Chạy thuật toán Apriori
frequent_itemsets = apriori(df, min_support=minSupport, use_colnames=True)
print("Tập mục phổ biến:")
print(frequent_itemsets)
# 4. Sinh luật kết hợp
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minConfidence)
print("\nLuật kết hợp:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

Tập mục phổ biến:
      support                       itemsets
0    0.036344                    (Alfajores)
1    0.016059                     (Baguette)
2    0.005071                     (Bakewell)
3    0.327205                        (Bread)
4    0.040042                      (Brownie)
..        ...                            ...
109  0.007184      (Coffee, Bread, Sandwich)
110  0.007396           (Coffee, Bread, Tea)
111  0.006867  (Coffee, Hot chocolate, Cake)
112  0.010037            (Coffee, Cake, Tea)
113  0.005388        (Coffee, Tea, Sandwich)

[114 rows x 2 columns]

Luật kết hợp:
            antecedents consequents   support  confidence      lift
0           (Alfajores)     (Bread)  0.010354    0.284884  0.870657
1           (Alfajores)    (Coffee)  0.019651    0.540698  1.130235
2           (Alfajores)       (Tea)  0.006762    0.186047  1.304393
3             (Brownie)     (Bread)  0.010777    0.269129  0.822508
4                (Cake)     (Bread)  0.023349    0.224822  0.68

## Thuật toán FBGrowth

In [15]:
frequent_itemsets = fpgrowth(df, min_support=minSupport, use_colnames=True)
print("Tập mục phổ biến:")
print(frequent_itemsets)
# 4. Tạo luật kết hợp
min_confidence = 0.7  # Ngưỡng confidence tối thiểu:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minConfidence)
# 5. In kết quả
print("Frequent Itemsets:")
print(frequent_itemsets.sort_values(by='support', ascending=False).head(10))
print("\nAssociation Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

Tập mục phổ biến:
      support                  itemsets
0    0.327205                   (Bread)
1    0.029054            (Scandinavian)
2    0.058320           (Hot chocolate)
3    0.054411                 (Cookies)
4    0.015003                     (Jam)
..        ...                       ...
109  0.018067           (Coffee, Scone)
110  0.008347              (Scone, Tea)
111  0.006550           (Coffee, Salad)
112  0.005177    (Coffee, Chicken Stew)
113  0.010882  (Coffee, Spanish Brunch)

[114 rows x 2 columns]
Frequent Itemsets:
     support         itemsets
6   0.478394         (Coffee)
0   0.327205          (Bread)
9   0.142631            (Tea)
18  0.103856           (Cake)
37  0.090016  (Coffee, Bread)
7   0.086107         (Pastry)
21  0.071844       (Sandwich)
8   0.061807      (Medialuna)
2   0.058320  (Hot chocolate)
82  0.054728   (Coffee, Cake)

Association Rules:
               antecedents      consequents   support  confidence      lift
0                 (Coffee)       